In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib
from mendeleev import element
from matplotlib import pyplot as plt
from qmpy.analysis.miedema import Miedema
from ase.data import atomic_names, atomic_numbers
%matplotlib inline

In [2]:
si = element('Pt')
econf = si.econf
print(econf)
re.findall(r'[spdf](\d*)\b', econf)

[Xe] 4f14 5d9 6s


['14', '9', '']

In [3]:
file_path = '../dataset/2021Yan-SP-HEA'
data_file = '2021Yan_dataset.csv'
bulk_modulus_data = 'bulk_modulus.csv'
df = pd.read_csv(f'{file_path}/{data_file}')
df.drop('Phase', inplace=True, axis=1)
df_k = pd.read_csv(f'{file_path}/{bulk_modulus_data}', header=None, 
                   names=['atomic_number', 'bulk_modulus'])
k_Ga = pd.DataFrame({'atomic_number': [31, 40, 32, 76, 49, 38, 43], 
                     'bulk_modulus': [56, 91, 74, 373, 40, 14.9, 157]},)
df_k = pd.concat([df_k, k_Ga], ignore_index=True)
df_k.reset_index()
df_k.head()

,atomic_number,bulk_modulus
0,3,9.0
1,4,129.0
2,5,321.0
3,6,34.0
4,11,6.0


In [4]:
df_k.to_csv(f'{file_path}/bulk_modulus_added.csv', index=False)

**Note**:

add Ga data: 44 to 68, avg: 56 GPa
add Zr data: 91 GPa
add Ge data: 70--78, avg 74 GPa
add Ge data: 358--388, avg 373 GPa
add In data: 30--50, avg 40 GPa
add Sr data from chatGPT: 14.9 GPa

- No bulk modulus data for Ga (Gallium, 31), added that from additional source: https://www.azom.com/properties.aspx?ArticleID=1132
- No Zr (40), add from source: https://material-properties.org/zirconium-mechanical-properties-strength-hardness-crystal-structure/#:~:text=The%20bulk%20modulus%20of%20elasticity%20of%20Zirconium%20is%2091%20GPa.
- No Ge (Germanium, 32), add from source: https://www.azom.com/properties.aspx?ArticleID=1837
- No Os (Osmium, 76), add from source:https://www.azom.com/properties.aspx?ArticleID=1842
- Other unavailable elements data are retrieved from chatGPT.

In [5]:
df.rename(columns={'Alloys':'Composition', 'Class': 'Is_SP'}).sample(5)

,Composition,Is_SP
1673,MoNbTiV0.5Zr,1
1635,Th0.25Zr0.75,1
1204,Al0.2CoCrFeNiTi0.5,1
388,Al1Co0.29Cr1Fe1Mo0.5Ni1,0
1041,Ni0.9W0.1,1


In [6]:
regex = fr'([A-Z][a-z]*)(\d*\.*\d*?(?=\D|$))'
df['alloy_sep'] = df['Alloys'].str.findall(regex)
df['alloy_sep'] = df['alloy_sep'].apply(lambda m: [(x, float(y)) if y else (x, 1) for x, y in m])
df['normalizer'] = df['alloy_sep'].apply(lambda m: sum([_[1] for _ in m]))
df.sample(5)

,Alloys,Class,alloy_sep,normalizer
65,Al0.348Co1Cr1Fe1Ni1,0,"[(Al, 0.348), (Co, 1.0), (Cr, 1.0), (Fe, 1.0),...",4.348
1180,Co1Fe1Ni1Pd1,1,"[(Co, 1.0), (Fe, 1.0), (Ni, 1.0), (Pd, 1.0)]",4.000
1333,Os0.25Ru0.75,1,"[(Os, 0.25), (Ru, 0.75)]",1.000
956,Ca1Nd1,0,"[(Ca, 1.0), (Nd, 1.0)]",2.000
1290,AlCoCrCuFeNiTiV,1,"[(Al, 1), (Co, 1), (Cr, 1), (Cu, 1), (Fe, 1), ...",8.000


In [7]:
all_elements = set()
for alloy_sep in df['alloy_sep']:
    for ele, _ in alloy_sep:
        if ele not in all_elements:
            all_elements.add(ele)
len(all_elements)

70

In [8]:
look_up_dict = {}
for ele in all_elements:
    res = {}
    melting_point = element(ele).melting_point
    res['tm'] = melting_point if isinstance(melting_point, float) else np.mean(list(melting_point.values()))
    res['tm'] = round(res['tm'], 2)
    econf = element(ele).econf
    nval_list = re.findall(r'[spdf](\d*)\b', econf)
    nval = sum(int(_) if _ else 1 for _ in nval_list)
    res['vac'] = nval
    res['vm'] = element(ele).atomic_volume
    res['chi'] = element(ele).en_pauling # pauling electronegativity
    res['ar'] = element(ele).atomic_radius # atomic radius
    look_up_dict[ele] = res
# look_up_dict

In [9]:
# For Tb and Yb, electronegativity not available, using web data
look_up_dict['Yb']['chi'] = 1.1
look_up_dict['Tb']['chi'] = 1.2
look_up_dict['Eu']['chi'] = 1.2
# look_up_dict

In [10]:
import pickle
with open('../pkl-files/look_up_dict.pkl', 'wb') as pf:
    pickle.dump(look_up_dict, pf)

In [11]:
### Feature Engineering
def melting_temperature(row):
    """Weighted average of element melting temperatures"""
    alloy_sep = row['alloy_sep']
    z = row['normalizer']
    Tm = 0
    for ele, n in alloy_sep:
        Tm += look_up_dict[ele]['tm'] * n / z
    return Tm

def valence_electron_concentration(row):
    """Weighted average of valence electron concentration"""
    alloy_sep = row['alloy_sep']
    z = row['normalizer']
    vac = 0
    for ele, n in alloy_sep:
        vac += look_up_dict[ele]['vac'] * n / z
    return vac

def molar_volume(row):
    """Weighted average of molar volume"""
    alloy_sep = row['alloy_sep']
    z = row['normalizer']
    vm = 0
    for ele, n in alloy_sep:
        vm += look_up_dict[ele]['vm'] * n / z
    return vm


def bulk_modulus(row):
    """Weighted average of bulk modulus"""
    alloy_sep = row['alloy_sep']
    z = row['normalizer']
    k = 0
    for ele, n in alloy_sep:
        val = df_k.loc[df_k['atomic_number'] == atomic_numbers[ele]]['bulk_modulus']
        val = val.values[0]
#         print(ele, val, alloy_sep, atomic_numbers[ele])
        k += val * n / z
    return k


In [12]:
df['k'] = df.apply(lambda row: bulk_modulus(row), axis=1)
df.sample(5)

,Alloys,Class,alloy_sep,normalizer,k
1022,Co0.75Fe0.25,1,"[(Co, 0.75), (Fe, 0.25)]",1.0,176.50
1014,Cd0.5Mg0.5,1,"[(Cd, 0.5), (Mg, 0.5)]",1.0,42.50
1583,Nb1Ta1Ti1V1W1,1,"[(Nb, 1.0), (Ta, 1.0), (Ti, 1.0), (V, 1.0), (W...",5.0,189.60
1587,Au0.85Pt0.15,1,"[(Au, 0.85), (Pt, 0.15)]",1.0,220.65
721,Cr1Hf1Nb1Ti1Zr1,0,"[(Cr, 1.0), (Hf, 1.0), (Nb, 1.0), (Ti, 1.0), (...",5.0,127.60


In [13]:
df['vm'] = df.apply(lambda row: molar_volume(row), axis=1)
df['tm'] = df.apply(lambda row: melting_temperature(row), axis=1)
df['vac'] = df.apply(lambda row: valence_electron_concentration(row), axis=1)
df.sample(5)

,Alloys,Class,alloy_sep,normalizer,k,vm,tm,vac
197,Co1Cu1Fe1Mn1Ni1Sn0.05W1,0,"[(Co, 1.0), (Cu, 1.0), (Fe, 1.0), (Mn, 1.0), (...",6.05,181.140496,7.476860,1965.505041,10.859504
484,Al0.5Co1Cr1Cu1Fe1Ni1Si1.4,0,"[(Al, 0.5), (Co, 1.0), (Cr, 1.0), (Cu, 1.0), (...",6.90,145.449275,8.213043,1691.900725,7.405797
895,Fe1Yb1,0,"[(Fe, 1.0), (Yb, 1.0)]",2.00,99.500000,15.950000,1454.150000,12.000000
1087,Al0.25CoFeNi,1,"[(Al, 0.25), (Co, 1), (Fe, 1), (Ni, 1)]",3.25,168.000000,7.046154,1704.866923,8.538462
828,Ag1Pb1,0,"[(Ag, 1.0), (Pb, 1.0)]",2.00,72.500000,14.300000,917.770000,19.500000


In [14]:
R = 8.314 # J K^-1 mol^-1
def mixing_entropy(row):
    """Mixing entropy"""
    alloy_sep = row['alloy_sep']
    z = row['normalizer']
    delta_s_mix = 0
    for ele, n in alloy_sep:
        c = n/z
        delta_s_mix += -c * np.log(c+np.finfo(np.float32).eps)
    delta_s_mix *= R
    return delta_s_mix

def pauling_eletronegativitiy_diff(row):
    """Pauling electronegativity difference"""
    alloy_sep = row['alloy_sep']
    z = row['normalizer']
    avg_chi = 0 
    for ele, n in alloy_sep:
        c = n/z
        avg_chi += c * look_up_dict[ele]['chi']
    delta_chi = 0
    for ele, n in alloy_sep:
        c = n / z
        delta_chi += c * (look_up_dict[ele]['chi'] - avg_chi) ** 2
    delta_chi = np.sqrt(delta_chi)
    return delta_chi

def atomic_size_diff(row):
    """Atomic size difference"""
    alloy_sep = row['alloy_sep']
    z = row['normalizer']
    avg_r = 0 
    for ele, n in alloy_sep:
        c = n/z
        avg_r += c * look_up_dict[ele]['ar']
    delta = 0
    for ele, n in alloy_sep:
        c = n / z
        delta += c * (1-look_up_dict[ele]['ar']/avg_r) ** 2
    delta = 100*np.sqrt(delta)
    return delta

In [15]:
df['delta_s_mix'] = df.apply(lambda row: mixing_entropy(row), axis=1)
df['delta_chi'] = df.apply(lambda row: pauling_eletronegativitiy_diff(row), axis=1)
df['delta'] = df.apply(lambda row: atomic_size_diff(row), axis=1)
df.sample(5)

,Alloys,Class,alloy_sep,normalizer,k,vm,tm,vac,delta_s_mix,delta_chi,delta
627,Co1Fe1Mn1Ti1V1Zr2,0,"[(Co, 1.0), (Fe, 1.0), (Mn, 1.0), (Ti, 1.0), (...",7.0,130.857143,9.762857,1925.578571,5.857143,14.531769,0.200845,5.567764
1431,Re0.5Tc0.5,1,"[(Re, 0.5), (Tc, 0.5)]",1.0,263.500000,8.675000,2944.150000,14.000000,5.762824,0.100000,0.000000
652,As1Cu1,0,"[(As, 1.0), (Cu, 1.0)]",2.0,80.500000,10.100000,1223.960000,13.000000,5.762824,0.140000,8.000000
740,Cr1Se1,0,"[(Cr, 1.0), (Se, 1.0)]",2.0,85.000000,11.865000,1326.850000,11.000000,5.762824,0.445000,9.803922
911,Ga1Na1,0,"[(Ga, 1.0), (Na, 1.0)]",2.0,31.000000,17.750000,336.925000,7.000000,5.762824,0.440000,16.129032


In [16]:
def mixing_enthalpy(row):
    """Mixing enthalpy using the Miedema's model based on the code:
    https://github.com/wolverton-research-group/qmpy/blob/master/qmpy/analysis/miedema.py
    Note that the module returns the formation enthalpy, you need to get rid of
    single-element terms to find the mixing enthalpy."""
    alloy_sep = row['alloy_sep']
    z = row['normalizer']
    delta_h_mix = 0
    for i, (ele_a, n_a) in enumerate(alloy_sep):
        for ele_b, n_b in alloy_sep[i+1:]:
            obj = Miedema({ele_a:1, ele_b:1})
            try:
                delta_H_ab = obj.H_mix
            except AttributeError:
                delta_H_ab = 0
            delta_h_mix += delta_H_ab * n_a * n_b / (z**2)
    delta_h_mix *= 4
    return delta_h_mix

In [17]:
df['delta_h_mix'] = df.apply(lambda row: mixing_enthalpy(row), axis=1)
df.sample(5)

,Alloys,Class,alloy_sep,normalizer,k,vm,tm,vac,delta_s_mix,delta_chi,delta,delta_h_mix
1100,CoCrFeMnNiV0.25,1,"[(Co, 1), (Cr, 1), (Fe, 1), (Mn, 1), (Ni, 1), ...",5.25,160.904762,7.068095,1819.530952,7.857143,14.335343,0.138099,1.794871,-8.228571
1170,Ni0.75Pt0.25,1,"[(Ni, 0.75), (Pt, 0.25)]",1.00,191.750000,7.225000,1806.450000,13.500000,4.675252,0.125574,0.000000,-5.400000
1602,MoNbTaTi0.25W,1,"[(Mo, 1), (Nb, 1), (Ta, 1), (Ti, 0.25), (W, 1)]",4.25,220.529412,10.183529,3084.326471,12.000000,12.707659,0.249523,2.979794,-9.018685
1005,Th0.75Zr0.25,1,"[(Th, 0.75), (Zr, 0.25)]",1.00,61.750000,18.375000,2049.150000,4.000000,4.675252,0.012990,6.230399,5.400000
127,Al40Co60Cr60Fe60Mn60Ni60,0,"[(Al, 40.0), (Co, 60.0), (Cr, 60.0), (Fe, 60.0...",340.00,151.000000,7.356471,1699.246471,7.411765,14.818060,0.139348,3.501741,-21.371626


In [18]:
df.to_csv(f'{file_path}/2021Yan_feature_engineered.csv', index=False)